# Алгоритмы SVD и ALS. Практика

Продолжим работать с датасетом Netflix.

Для экономии времени сразу возьмем урезанный датасет data_fin_half_sample.csv. Он сформирован как подвыборка из 5000 случайных кастомеров и 1000 фильмов из изходного набора данных.


In [1]:
!ls -la ../data/ | grep data_fin

-rw-r--r--  1 501 dialout 1806696634 Aug 22 17:26 data_fin.csv
-rw-r--r--  1 501 dialout  566312581 Aug 22 17:56 data_fin_half.csv
-rw-r--r--  1 501 dialout    9344707 Aug 24 20:45 data_fin_half_sample.csv


Загружаем наши данные:

In [2]:
import pandas as pd
df = pd.read_csv('../data/data_fin_half_sample.csv', sep=';')

Подгружаем библиотеку surprise, которую мы уже установили ранее:

In [5]:
import surprise
from surprise import Reader, Dataset

И подгружаем в surprise выбранный нами датасет:

In [6]:
reader = Reader(rating_scale=(0.5,5.0))
data = Dataset.load_from_df(df[['Cust_Id','Movie_Id','Rating']],reader)

Добавляем алгоритм SVD:

In [8]:
from surprise import SVD

Задаём наш алгоритм и формируем обучающую выборку:

In [9]:
svd = SVD()
trainingSet = data.build_full_trainset()

Тренируем наш алгоритм:

In [10]:
svd.fit(trainingSet)

Далее мы уже можем получить предсказание:

In [11]:
testSet=trainingSet.build_testset()
predictions=svd.test(testSet)

Подгружаем названия фильмов:

In [13]:
titles = pd.read_csv('../data/movie_titles.csv', encoding = "ISO-8859-1", 
                     header = None, 
                     names = ['Movie_Id', 'Year', 'Name'])

Далее можем задать функцию для определения топ-3 рекомендаций:

In [14]:
from collections import defaultdict
 
def get_top3_recommendations(predictions, topN = 3):
     
    top_recs = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_recs[uid].append((iid, est))
     
    for uid, user_ratings in top_recs.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_recs[uid] = user_ratings[:topN]
     
    return top_recs

Обрабатываем наше предсказание:

In [15]:
top3_recommendations = get_top3_recommendations(predictions)

С помощью следующей функции переведем тексты фильмов в удобочитаемый вид, то есть раскодируем заглавия фильмов. 

In [18]:
import numpy as np
def print_recs(i):
    for (a, b) in top3_recommendations[i]:
        print(titles[titles.Movie_Id == a]['Name'].values[0], np.round(b,2))

С помощью этой функции выведем рекомендации для случайного пользователя:

In [19]:
i = np.random.choice(list(top3_recommendations.keys()))

print_recs(i)

The Shawshank Redemption: Special Edition 4.94
The Green Mile 4.6
Saving Private Ryan 4.6


Давайте посмотрим, что смотрел этот человек, и выберем из нашего датасета те фильмы, которые этот человек оценил на 5. 

In [20]:
films = data.df[(data.df.Cust_Id == i) & (data.df.Rating == 5)]['Movie_Id'].values
titles[titles.Movie_Id.isin(films)]['Name'].values

array(["What's Eating Gilbert Grape", 'Office Space',
       'The Shawshank Redemption: Special Edition', 'Beetlejuice',
       'Dead Poets Society', 'The Big Lebowski', 'The Green Mile',
       'Indiana Jones and the Last Crusade', 'Saving Private Ryan',
       'Malcolm X'], dtype=object)